In [1]:
import sys
# sys.path.remove('/opt/ros/kinetic/lib/python2.7/dist-packages')
import numpy as np
import cv2
import matplotlib.pyplot as plt
import glob as glob
import os



In [2]:
img_defects_path = glob.glob(os.path.join('/home/hitech/Downloads/YE358311_Fender_apron/YE358311_defects','*/*.jpg'))
img_healthy_path = glob.glob(os.path.join('/home/hitech/Downloads/YE358311_Fender_apron/YE358311_Healthy','*.jpg'))
img_size = 224


In [3]:
def img_preprocessing(img):
    img = cv2.resize(img, (img_size,img_size))
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return img
    

def image_labels(path, label=0):
    imgs = []
    labels = []
    
    for i in path:
        img = cv2.imread(i)
        img = img_preprocessing(img)
#         break
        imgs.append(img)    
        labels.append(label)
    
    return imgs, labels

In [4]:
img_healthy, labels_healthy  = image_labels(img_healthy_path,0)
img_defective, labels_defective = image_labels(img_defects_path,1)

final_images = img_healthy + img_defective
labels = labels_healthy + labels_defective

X = np.array(final_images)
X = X.reshape(X.shape[0],img_size,img_size,3)
Y = np.array(labels)

In [5]:
print(np.shape(img_healthy))

(139, 224, 224, 3)


In [8]:
from sklearn.model_selection import train_test_split
X_train,X_test, Y_train, Y_test =train_test_split(X, Y, test_size=0.1, random_state=42)



In [118]:
# classifier = tensorflow.keras.Sequential()
# classifier.add(Conv2D(64,  (3, 3), padding='same', input_shape = (img_size, img_size,3)))
# classifier.add(BatchNormalization())
# classifier.add(Activation('relu'))
# classifier.add(MaxPooling2D(pool_size = (2, 2)))

# classifier.add(Conv2D(128, (3, 3), padding='same'))
# classifier.add(Conv2D(128, (3, 3), padding='same'))
# classifier.add(Conv2D(128, (3, 3), padding='same'))

# classifier.add(BatchNormalization())
# classifier.add(Activation('relu'))
# classifier.add(MaxPooling2D(pool_size = (2, 2)))


# classifier.add(Conv2D(256, (3, 3), padding='same'))
# classifier.add(Conv2D(256, (3, 3), padding='same'))
# classifier.add(Conv2D(256, (3, 3), padding='same'))

# classifier.add(BatchNormalization())
# classifier.add(Activation('relu'))
# classifier.add(MaxPooling2D(pool_size = (2, 2)))


# classifier.add(Conv2D(512, (3, 3), padding = 'same'))
# classifier.add(Conv2D(512, (3, 3), padding='same'))
# classifier.add(Conv2D(512, (3, 3), padding='same'))

# classifier.add(BatchNormalization())
# classifier.add(Activation('relu'))
# classifier.add(MaxPooling2D(pool_size = (2, 2)))


# classifier.add(Flatten())

# classifier.add(Dense(units = 4096, activation = 'relu'))#4096
# classifier.add(Dense(units = 4096, activation = 'relu'))#4096
# classifier.add(Dense(units = 1, activation = 'sigmoid'))

# classifier.summary()

# classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_38 (Conv2D)           (None, 128, 128, 64)      1792      
_________________________________________________________________
batch_normalization_v1_30 (B (None, 128, 128, 64)      256       
_________________________________________________________________
activation_29 (Activation)   (None, 128, 128, 64)      0         
_________________________________________________________________
max_pooling2d_28 (MaxPooling (None, 64, 64, 64)        0         
_________________________________________________________________
conv2d_39 (Conv2D)           (None, 64, 64, 128)       73856     
_________________________________________________________________
conv2d_40 (Conv2D)           (None, 64, 64, 128)       147584    
_________________________________________________________________
conv2d_41 (Conv2D)           (None, 64, 64, 128)       147584    
__________

In [13]:
filepath="weights.defective_parts_BGM_2.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
# history = classifier.fit_generator(datagen.flow(X_train, Y_train, batch_size=batch_size),
#                      epochs=epochs,validation_data=(X_test, Y_test), callbacks=callbacks_list)

# model.fit_generator(
#         train_generator,
#         steps_per_epoch=2000,
#         epochs=50,
#         validation_data=validation_generator,
#         validation_steps=800)


Epoch 1/10


In [20]:
epochs = 40
batch_size = 4
testsplit = .2
targetx = 224
targety = 224
learning_rate = 0.0001
classes = 1
# seed = random.randint(1, 1000)

In [11]:
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image                  
import numpy as np

from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.utils import *
from keras.callbacks import *
from keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input, decode_predictions

Using TensorFlow backend.


In [12]:
from keras.applications import MobileNet
from keras.applications.mobilenet import preprocess_input

In [14]:
datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        vertical_flip=True,
        horizontal_flip=True,
        rotation_range=90
)
datagen.fit(X_train)
datagen.fit(X_test)

In [15]:
base_model = MobileNetV2(include_top=False, weights='imagenet', input_shape=(targetx, targety, 3))

Instructions for updating:
Colocations handled automatically by placer.


In [16]:
base_model.layers[-1].outbound_nodes = []
x = base_model.output
# x = GlobalAveragePooling2D()(x)
x = Flatten()(x)
x = Dense(512,activation='relu')(x)
x = BatchNormalization()(x)
# x = Dense(1024,activation='relu')(x)
# x = BatchNormalization()(x)

predictions = Dense(classes, activation='sigmoid')(x)
model = Model(base_model.input, predictions)


In [17]:
optimizer = Adam(lr=learning_rate)
loss = "binary_crossentropy"

In [18]:
for layer in model.layers:
    layer.trainable = True

In [19]:
model.compile(optimizer=optimizer,
              loss=loss,
              metrics=["accuracy"])

# model.summary()

In [21]:
history = model.fit_generator(datagen.flow(X_train, Y_train, batch_size=batch_size),
                     epochs=epochs,validation_data=(X_test, Y_test), callbacks=callbacks_list, 
                              steps_per_epoch=len(X_train) / batch_size)


Instructions for updating:
Use tf.cast instead.
Epoch 1/40
56/56 [==============================] - 105s 2s/step - loss: 1.1666 - acc: 0.5893 - val_loss: 1.6756 - val_acc: 0.4800

Epoch 00001: val_acc improved from -inf to 0.48000, saving model to weights.defective_parts_BGM_2.hdf5
Epoch 2/40
56/56 [==============================] - 94s 2s/step - loss: 1.1284 - acc: 0.6472 - val_loss: 2.3304 - val_acc: 0.4800

Epoch 00002: val_acc did not improve from 0.48000
Epoch 3/40
56/56 [==============================] - 93s 2s/step - loss: 1.1705 - acc: 0.5984 - val_loss: 1.4526 - val_acc: 0.4800

Epoch 00003: val_acc did not improve from 0.48000
Epoch 4/40
56/56 [==============================] - 95s 2s/step - loss: 0.6719 - acc: 0.7321 - val_loss: 0.9921 - val_acc: 0.4400

Epoch 00004: val_acc did not improve from 0.48000
Epoch 5/40
56/56 [==============================] - 93s 2s/step - loss: 0.7539 - acc: 0.6830 - val_loss: 1.0714 - val_acc: 0.4800

Epoch 00005: val_acc did not improve from 0